## SQL ##

*En esta fase del proceso de ETL  vamos a crear paso a paso una base de datos en sql desde python, para despúes crear las tablas con sus PK y sus FK.*

*Una vez todo esto este creado procedemos a poblarla desde python con nuestro df limpio y actualizado.*

Este es el codigo usado en sql para crear nuestra base de datos. *Este codigo se puede usar para crear la base de datos directamente desde SQL,pero vamos a crearla desde este entorno*

In [1]:
import pandas as pd
import mysql.connector
import numpy as np

Lectura y preparado de los distindos CSV

In [ ]:
df = pd.read_csv("final_cleaned_sentiment.csv")
df.head(3)
df.shape

In [ ]:
df_service = pd.read_csv("df_servicios_final_cleaned.csv")
df_service.head(3)

In [ ]:
df3 = pd.merge(left = df, right = df_service, on = "urls")[["urls", "record_id", "services", "category"]]

df3.head(3)

In [ ]:
tabla_service = pd.DataFrame(data = [[num, v] for num, v in enumerate(df3["services"].unique(), start = 1)],
                             columns = ["service_id", "service"])
tabla_service.head(3)

In [ ]:
tabla_category = pd.DataFrame(data = [[num, v] for num, v in enumerate(df3["category"].unique(), start = 1)],
                             columns = ["category_id", "category"])
tabla_category.head(3)

In [ ]:
map_category = {k : v for v, k in tabla_category.values} 
map_service = {k : v for v, k in tabla_service.values}

In [ ]:
tabla_category_service = df3[["services", "category"]]

tabla_category_service["category"] = tabla_category_service["category"].map(map_category)
tabla_category_service["services"] = tabla_category_service["services"].map(map_service)

tabla_category_service

In [ ]:
tabla_service_record = df3[["record_id", "services"]]

tabla_service_record["services"] = tabla_service_record["services"].map(map_service)

Este es el codigo para crear desde python la base de datos en sql.

*Correr solo una vez*

In [ ]:
db = mysql.connector.connect(
     host="localhost",
     user="root",
     password="contraseña123"#Aqui teneis que poner vuestra contraeña.
)

cursor = db.cursor()

# Crear la base de datos si no existe
cursor.execute('CREATE DATABASE IF NOT EXISTS AIRBNB')

# Conectar a la base de datos recién creada
db.database = 'AIRBNB'

# Crear la tabla Hosting
cursor.execute('''CREATE TABLE IF NOT EXISTS Hosting (
        record_id BIGINT,
        titles TEXT,
        property_types TEXT,
        host_name TEXT,
        PRIMARY KEY (record_id));''')

# Crear la tabla Description_
cursor.execute('''CREATE TABLE IF NOT EXISTS Description_ (
        id BIGINT,
        prices_per_night INT,
        check_in_hour TIME,
        check_out_hour TIME,
        total_hours_checkin INT,
        cleaning_fee INT,
        maximum_guests INT,
        camas INT,
        baños INT,
        dormitorios INT,
        FOREIGN KEY (id) REFERENCES Hosting(record_id));''')

# Crear la tabla Ratings
cursor.execute('''CREATE TABLE IF NOT EXISTS Ratings (
        record_id BIGINT,
        ratings FLOAT,
        num_reviews FLOAT,
        sentiment TEXT,
        FOREIGN KEY (record_id) REFERENCES Hosting(record_id));''')

#Creamos la tabla de Services_
cursor.execute('''CREATE TABLE IF NOT EXISTS Services_ (
        service_id INT,
        service TEXT,
        PRIMARY KEY (service_id));''')
#Creamos la tabla de Service_ids
cursor.execute('''CREATE TABLE IF NOT EXISTS Services_Hosting (
        service_id INT,
        record_id BIGINT,
        FOREIGN KEY (service_id) REFERENCES Services_(service_id),
        FOREIGN KEY (record_id) REFERENCES Hosting (record_id));''')   
#Creamos la tabla de Category
cursor.execute('''CREATE TABLE IF NOT EXISTS Category (
        category_id INT,
        category TEXT,
        PRIMARY KEY (category_id));''')

#Creamos la tabla de Category_ids
cursor.execute('''CREATE TABLE IF NOT EXISTS Category_Services (
        service_id INT,
        category_id INT,
        FOREIGN KEY (service_id) REFERENCES Services_(service_id),
        FOREIGN KEY (category_id) REFERENCES Category(category_id));''')   

# Cerrar el cursor y la conexión
cursor.close()
db.close()

Este es el codigo para la subida de nuestros CSV a la base de datos (airbnb)

In [ ]:
def insert_to_table(df, database, table_name):

    db = mysql.connector.connect(host="localhost",
                                 user="root",
                                 password="contraseña123",#Aqui teneis que poner vuestra contraeña.
                                 database=database,
                                 consume_results = True)
    
    cursor = db.cursor()
    
    # Seleccionamos las columnas de la tabla, omitiendo la Primary Key
    cursor.execute(f"SELECT * FROM {table_name} LIMIT 0;")
    column_names = cursor.column_names
    
    # Aseguramos que todas las filas del DataFrame tengan el mismo número de columnas que en la tabla
    if len(df.columns) != len(column_names):
        raise ValueError("El número de columnas en el DataFrame no coincide con el número de columnas en la tabla de la base de datos.")
    
    # Preparar la consulta de inserción
    insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s' for _ in column_names])})"
    
    # Convertir las filas del DataFrame a tuplas
    values = [tuple(row) for row in df[list(column_names)].values]
    
    # Ejecutar la consulta de inserción
    cursor.executemany(insert_query, values)
    
    # Confirmar los cambios en la base de datos
    db.commit()
    
    print(f"Añadidas: {cursor.rowcount} filas")

    # Vaciamos el cursor
    cursor.close()
    db.close()

Aqui procedemos a la subida de las distintas tablas.

In [ ]:
database = "AIRBNB"

Hosting

In [ ]:
tabla_hosting = df[["record_id", "titles", "property_types", "host_name"]].drop_duplicates(subset = ["record_id"])
tabla_hosting = tabla_hosting.replace([np.nan], [None])
insert_to_table(df = tabla_hosting, database = database, table_name = "Hosting")

services_

In [ ]:
insert_to_table(df = tabla_service, database = database, table_name = "services_")

category

In [ ]:
insert_to_table(df = tabla_category, database = database, table_name = "category")

Description_

In [ ]:
tabla_description_ = df[["record_id", "prices_per_night", "check_in_hour", "check_out_hour", "total_hours_checkin",
                    "cleaning_fee", "maximum_guests", "camas", "baños", "dormitorios"]]
tabla_description_.columns = ["id", "prices_per_night", "check_in_hour", "check_out_hour", "total_hours_checkin",
                    "cleaning_fee", "maximum_guests", "camas", "baños", "dormitorios"]

tabla_description_ = tabla_description_.replace([np.nan], [None])

insert_to_table(df = tabla_description_, database = database, table_name = "description_")

Ratings

In [ ]:
tabla_ratings = df[["record_id", "ratings", "num_reviews", "sentiment"]]
insert_to_table(df = tabla_ratings, database = database, table_name = "ratings")

services_hosting

In [ ]:
tabla_service_record["services"] = tabla_service_record["services"].astype("str")
tabla_service_record.columns = ["record_id", "service_id"]
insert_to_table(df = tabla_service_record, database = database, table_name = "services_hosting")

category_services

In [ ]:
tabla_category_service.columns = ["service_id", "category_id"]
tabla_category_service["service_id"] = tabla_category_service["service_id"].astype("str")
insert_to_table(df = tabla_category_service, database = database, table_name = "category_services")

# Actualizar base de datos

**Inserta solo los datos que no existan en la base de datos.**

*ruta_csv*: Ruta del archivo CSV que contiene los nuevos datos.

*conexion_db*: Objeto de conexión a la base de datos MySQL (usando mysql.connector).

*tabla*: Nombre de la tabla en la que se insertarán los datos.

*columnas_unicas*: Lista de columnas que definen la unicidad de los registros. 

In [ ]:
def actualizar_base_datos(ruta_csv, conexion_db, tabla, columnas_unicas):
    
    # Crear cursor
    cursor = conexion_db.cursor(dictionary=True)

    # Cargar datos desde el archivo CSV
    nuevos_datos = pd.read_csv(ruta_csv)

    # Iterar sobre cada fila de datos nuevos
    for _, fila in nuevos_datos.iterrows():
        # Construir la condición para la consulta de verificación
        condicion = " AND ".join([f"{col} = %s" for col in columnas_unicas])
        valores_condicion = [fila[col] for col in columnas_unicas]
        query = f"SELECT COUNT(*) as conteo FROM {tabla} WHERE {condicion}"
        
        # Ejecutar la consulta de verificación
        cursor.execute(query, valores_condicion)
        resultado = cursor.fetchone()
        
        # Si no hay registros duplicados, insertar la fila
        if resultado['conteo'] == 0:
            columnas = ", ".join(fila.index)
            valores = ", ".join(["%s"] * len(fila))
            insert_query = f"INSERT INTO {tabla} ({columnas}) VALUES ({valores})"
            cursor.execute(insert_query, list(fila))
            conexion_db.commit()
            print(f"Datos insertados: {fila[columnas_unicas].to_dict()}")
        else:
            print(f"Duplicado encontrado, datos no insertados: {fila[columnas_unicas].to_dict()}")

    # Cerrar el cursor al finalizar
    cursor.close()

Ejemplo

In [ ]:
"""
db = mysql.connector.connect(
     host="localhost",
     user="root",
     password="contraseña123"
 )
db.database = 'AIRBNB'
actualizar_base_datos('ruta.csv', db, 'nombre_tabla', ['columnas'])
"""

'\ndb = mysql.connector.connect(\n     host="localhost",\n     user="root",\n     password="contraseña123"\n )\ndb.database = \'AIRBNB\'\nactualizar_base_datos(\'ruta.csv\', db, \'nombre_tabla\', [\'columnas\'])\n'

# Extracción de datos

In [ ]:
def extraer_datos_mysql(host, user, password, database, query):
    # Conectarse a la base de datos
    db = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    
    # Crear cursor para ejecutar la consulta
    cursor = db.cursor()

    # Ejecutar la consulta
    cursor.execute(query)
    
    # Obtener los nombres de las columnas
    column_names = cursor.column_names
    print("Nombres de las columnas:", column_names)

    # Guardar y mostrar los datos de la tabla
    data = cursor.fetchall()
    for fila in data:
        print(fila)

    # Cerrar el cursor y la conexión
    cursor.close()
    db.close()


Hosting

In [ ]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBNB"
query = "SELECT * FROM Hosting;"

extraer_datos_mysql(host, user, password, database, query)

Nombres de las columnas: ('record_id', 'titles', 'property_types', 'host_name')
(72436, 'Disponemos de 19 apartamentos de 45 a 57m2 situados entre la planta baja y la 3ª. tienen las mismas características: 1 habitación, 1 baño, salón-cocina, con decoraciones diferentes. no todos recogidos en la galería de fotos. apartamentos estilosos delante de la playa.acceso a la piscina con medidas 4x5.75 y 3x2000171/000172/000182/011441/011442/011440/011443/011444/010962/002917/010961/005109/005102/010969/005112/005100/010968/01097/010999.el espaciolos apartamentos del lugaris beach presentan un diseño contemporáneo y cuentan con ventanales, conexión a internet de alta velocidad gratuita y cocina privada equipada con horno, cafetera nespresso y nevera. además, constan de sala de estar, tv de pantalla plana con canales internacionales, lavadora y secadora de ropa. también incluyen 1 paquete de bienvenida con productos básicos como café, té, leche, azúcar, sal y aceite. además de un botella de agua 

Description_

In [ ]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBNB"
query = "SELECT * FROM Description_;"

extraer_datos_mysql(host, user, password, database, query)

Nombres de las columnas: ('id', 'prices_per_night', 'check_in_hour', 'check_out_hour', 'total_hours_checkin', 'cleaning_fee', 'maximum_guests', 'camas', 'baños', 'dormitorios')
(1279040855576324107, 46, datetime.timedelta(seconds=61200), datetime.timedelta(seconds=39600), 7, 15, 1, 1, 1, 1)
(1265660833018089515, 100, datetime.timedelta(seconds=57600), datetime.timedelta(seconds=43200), 8, 35, 1, 1, 1, 1)
(1197637831227097242, 115, datetime.timedelta(seconds=54000), datetime.timedelta(seconds=39600), 9, 70, 2, 1, 1, 1)
(1123156827645522473, 49, datetime.timedelta(seconds=54000), datetime.timedelta(seconds=43200), 9, 0, 1, 1, 1, 1)
(31977858, 18, datetime.timedelta(seconds=54000), datetime.timedelta(seconds=39600), 9, 0, 1, 1, 1, 1)
(1268163818980729724, 147, datetime.timedelta(seconds=54000), datetime.timedelta(seconds=43200), 9, 0, 4, 2, 1, 2)
(1086018140861723540, 122, datetime.timedelta(seconds=57600), datetime.timedelta(seconds=43200), 8, 40, 3, 2, 1, 1)
(1278815157116605567, 142, d

Ratings

In [ ]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBNB"
query = "SELECT * FROM Ratings;"

extraer_datos_mysql(host, user, password, database, query)

Nombres de las columnas: ('record_id', 'ratings', 'num_reviews', 'sentiment')
(1279040855576324107, 0.0, 0.0, 'Positivo')
(1265660833018089515, 4.89, 0.0, 'Positivo')
(1197637831227097242, 5.0, 4.0, 'Positivo')
(1123156827645522473, 4.65, 23.0, 'Positivo')
(31977858, 4.34, 296.0, 'Positivo')
(1268163818980729724, 0.0, 0.0, 'Positivo')
(1086018140861723540, 5.0, 3.0, 'Positivo')
(1278815157116605567, 0.0, 0.0, 'Positivo')
(40577359, 4.47, 103.0, 'Positivo')
(22548858, 4.84, 0.0, 'Positivo')
(1277720203892360783, 0.0, 0.0, 'Positivo')
(1248794356730472452, 0.0, 0.0, 'Positivo')
(39788335, 4.81, 48.0, 'Positivo')
(1100921852236177943, 4.71, 34.0, 'Positivo')
(12634537, 4.75, 0.0, 'Positivo')
(1210293681811098298, 0.0, 0.0, 'Positivo')
(850684095339937556, 5.0, 5.0, 'Positivo')
(10038461, 4.87, 0.0, 'Positivo')
(1273185111265704381, 0.0, 0.0, 'Positivo')
(961442045210294120, 4.51, 81.0, 'Positivo')
(1109989, 4.8, 573.0, 'Positivo')
(549976913937241380, 4.46, 26.0, 'Positivo')
(80004, 4.66,

Services

In [ ]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBNB"
query = "SELECT * FROM Services_;"

extraer_datos_mysql(host, user, password, database, query)

Nombres de las columnas: ('service_id', 'service')
(1, 'secador de pelo')
(2, 'productos de limpieza')
(3, 'champú')
(4, 'gel de ducha')
(5, 'agua caliente')
(6, 'lavadora')
(7, 'perchas')
(8, 'ropa de cama')
(9, 'persianas o cortinas opacas')
(10, 'plancha')
(11, 'tendedero para ropa')
(12, 'espacio para guardar la ropa: armario')
(13, 'tv')
(14, 'aire acondicionado')
(15, 'calefacción central')
(16, 'botiquín')
(17, 'wifi')
(18, 'cocinacocina disponible para el uso de los huéspedes')
(19, 'frigorífico')
(20, 'microondas')
(21, 'congelador')
(22, 'horno')
(23, 'hervidor de agua')
(24, 'entrada independienteentrada por otra calle o edificio')
(25, 'lavandería cercana')
(26, 'patio o balcón')
(27, 'comedor al aire libre')
(28, 'ascensorel alojamiento o edificio dispone de un ascensor de al menos 132\xa0centímetros de profundidad cuya puerta mide como mínimo 81\xa0centímetros de ancho.')
(29, 'admite mascotaslos animales de asistencia siempre están permitidos')
(30, 'se permite dejar el 

Category

In [ ]:
host = "localhost"
user = "root"
password = "contraseña123"
database = "AIRBNB"
query = "SELECT * FROM Category;"

extraer_datos_mysql(host, user, password, database, query)

Nombres de las columnas: ('category_id', 'category')
(1, 'baño')
(2, 'dormitorio y lavandería')
(3, 'entretenimiento')
(4, 'calefacción y refrigeración')
(5, 'seguridad en el hogar')
(6, 'internet y oficina')
(7, 'cocina y comedor')
(8, 'características de la ubicación')
(9, 'exterior')
(10, 'aparcamiento e instalaciones')
(11, 'servicios')
(12, 'privacidad y seguridad')
(13, 'para familias')
